In [1]:
import pandas as pd

In [2]:
#extracting data
sales_df = pd.read_csv('Downloads/dataset.csv')

print (sales_df)

   order_id customer_id product_id  order_date  quantity  price_per_unit  \
0    ORD001     CUST008    PROD005  2023-09-24       3.0          411.75   
1    ORD002     CUST009    PROD019  2023-01-22       1.0          164.88   
2    ORD003     CUST004    PROD001  2023-04-10       2.0          279.17   
3    ORD004     CUST008    PROD019  2023-02-14       5.0          491.56   
4    ORD005     CUST004    PROD003  2023-08-28       4.0          236.42   
5    ORD006     CUST009    PROD005  2023-04-09       1.0          104.79   
6    ORD007     CUST008    PROD001  2023-03-24       6.0          126.96   
7    ORD008     CUST004    PROD003  2023-01-14       2.0          367.57   
8    ORD009     CUST008    PROD005  2023-05-18       7.0          210.82   
9    ORD010     CUST009    PROD019  2023-02-28       3.0          454.12   
10   ORD011     CUST004    PROD001  2023-06-21       8.0          187.95   
11   ORD012     CUST008    PROD003  2023-07-06       4.0          301.66   
12   ORD013 

In [3]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   order_id             25 non-null     object 
 1   customer_id          25 non-null     object 
 2   product_id           25 non-null     object 
 3   order_date           25 non-null     object 
 4   quantity             25 non-null     float64
 5   price_per_unit       25 non-null     float64
 6   discount_percentage  25 non-null     float64
 7   shipping_cost        25 non-null     float64
 8   payment_method       25 non-null     object 
 9   region               25 non-null     object 
dtypes: float64(4), object(6)
memory usage: 2.1+ KB


In [4]:
# 1. Convert order_date to datetime
sales_df['order_date'] = pd.to_datetime(sales_df['order_date'])
print("order_date converted to datetime successfully.")

# 2. Convert quantity to integer
sales_df['quantity'] = sales_df['quantity'].astype(int)
print("quantity converted to integer successfully.")

# 3. Drop discount_percentage
sales_df = sales_df.drop(columns=['discount_percentage'])
print("discount_percentage column dropped.")

# 4. Check for duplicates
total_duplicates = sales_df.duplicated().sum()
order_id_duplicates = sales_df['order_id'].duplicated().sum()

print(f"Total duplicate rows: {total_duplicates}")
print(f"Duplicate order_id values: {order_id_duplicates}")

# 5. lower case payment method
sales_df['payment_method'] = sales_df['payment_method'].str.lower()
print("payment_method converted to lowercase successfully.")

order_date converted to datetime successfully.
quantity converted to integer successfully.
discount_percentage column dropped.
Total duplicate rows: 0
Duplicate order_id values: 0
payment_method converted to lowercase successfully.


In [5]:
sales_df.describe()

,order_date,quantity,price_per_unit,shipping_cost
count,25,25.000000,25.000000,25.000000
mean,2023-05-20 01:55:12,3.960000,287.428000,16.390400
min,2023-01-14 00:00:00,1.000000,104.790000,12.780000
25%,2023-03-10 00:00:00,2.000000,177.450000,15.220000
50%,2023-05-18 00:00:00,3.000000,266.540000,15.710000
75%,2023-08-01 00:00:00,6.000000,399.770000,17.480000
max,2023-10-30 00:00:00,9.000000,491.560000,21.570000
std,NaN,2.457641,125.789602,2.652665


In [6]:
sales_df.head()

,order_id,customer_id,product_id,order_date,quantity,price_per_unit,shipping_cost,payment_method,region
0,ORD001,CUST008,PROD005,2023-09-24,3,411.75,12.78,cash on delivery,North
1,ORD002,CUST009,PROD019,2023-01-22,1,164.88,15.22,credit card,West
2,ORD003,CUST004,PROD001,2023-04-10,2,279.17,21.57,credit card,East
3,ORD004,CUST008,PROD019,2023-02-14,5,491.56,17.48,credit card,Central
4,ORD005,CUST004,PROD003,2023-08-28,4,236.42,15.71,credit card,North


In [8]:

# 1. Create total_price
sales_df['total_price'] =sales_df['quantity'] * sales_df['price_per_unit']
print ("total_price")

# 2. Create final_amount_paid
sales_df['final_amount_paid'] = sales_df['total_price']  + sales_df['shipping_cost']
print ("final_amount_paid")

# 3. Extract date features
sales_df['order_year'] = sales_df['order_date'].dt.year
sales_df['order_month'] = sales_df['order_date'].dt.month
sales_df['order_day'] = sales_df['order_date'].dt.day
sales_df['order_weekday'] = sales_df['order_date'].dt.day_name()
print("Date features")

# 4. Create weekend flag
sales_df['is_weekend'] = sales_df['order_weekday'].isin(['Saturday', 'Sunday'])
print("is_weekend")

# 5. Create order size category
sales_df['order_size'] = sales_df['quantity'].apply(
    lambda x: 'Small' if x <= 2 else ('Medium' if x <= 5 else 'Large'))
print("order_size")


total_price
final_amount_paid
Date features
is_weekend
order_size


In [9]:
# Profitability metrics per product
product_profitability = (
    sales_df.groupby('product_id')
            .agg({
                'total_price': 'sum',
                'shipping_cost': 'sum',
                'final_amount_paid': 'sum'
            })
            .reset_index()
)

# Profit = revenue - shipping
product_profitability['profit'] = (
    product_profitability['final_amount_paid'] - product_profitability['shipping_cost']
)

# Sort by profit
product_profitability = product_profitability.sort_values(by='profit', ascending=False)

print("Product profitability metrics created.")


Product profitability metrics created.


In [10]:
# Region-level aggregations
region_summary = (
    sales_df.groupby('region')
            .agg({
                'final_amount_paid': 'sum',
                'quantity': 'sum',
                'shipping_cost': 'sum'
            })
            .reset_index()
)

region_summary['avg_order_value'] = (
    region_summary['final_amount_paid'] / region_summary['quantity']
)

print("Region-level aggregations created.")


Region-level aggregations created.


In [11]:
# Weekend vs weekday sales
weekend_comparison = (
    sales_df.groupby('is_weekend')
            .agg({
                'final_amount_paid': 'sum',
                'quantity': 'sum',
                'order_id': 'count'
            })
            .reset_index()
)

weekend_comparison['day_type'] = weekend_comparison['is_weekend'].map({
    True: 'Weekend',
    False: 'Weekday'
})

print("Weekend vs weekday sales comparison created.")


Weekend vs weekday sales comparison created.


In [12]:
# Orders per day of month
orders_by_day = sales_df.groupby('order_day')['order_id'].count().reset_index()

highest_day = orders_by_day.loc[orders_by_day['order_id'].idxmax()]
least_day = orders_by_day.loc[orders_by_day['order_id'].idxmin()]

print("Highest order day:", highest_day['order_day'], "with", highest_day['order_id'], "orders")
print("Least order day:", least_day['order_day'], "with", least_day['order_id'], "orders")


Highest order day: 28 with 3 orders
Least order day: 1 with 1 orders


In [13]:
# Orders per month
orders_by_month = sales_df.groupby('order_month')['order_id'].count().reset_index()

highest_month = orders_by_month.loc[orders_by_month['order_id'].idxmax()]
least_month = orders_by_month.loc[orders_by_month['order_id'].idxmin()]

print("Highest order month:", highest_month['order_month'], "with", highest_month['order_id'], "orders")
print("Least order month:", least_month['order_month'], "with", least_month['order_id'], "orders")


Highest order month: 1 with 3 orders
Least order month: 10 with 1 orders


In [14]:
# Orders per size category
orders_by_size = sales_df.groupby('order_size')['order_id'].count().reset_index()

highest_size = orders_by_size.loc[orders_by_size['order_id'].idxmax()]
least_size = orders_by_size.loc[orders_by_size['order_id'].idxmin()]

print("Most common order size:", highest_size['order_size'], "with", highest_size['order_id'], "orders")
print("Least common order size:", least_size['order_size'], "with", least_size['order_id'], "orders")


Most common order size: Medium with 9 orders
Least common order size: Large with 7 orders


In [15]:
# Top products per order size
top_products_by_size = (
    sales_df.groupby(['order_size', 'product_id'])['order_id']
            .count()
            .reset_index(name='order_count')
)

# Sort within each size category
top_products_by_size = (
    top_products_by_size.sort_values(['order_size', 'order_count'], ascending=[True, False])
)

print("Top products per order size calculated.")


Top products per order size calculated.


In [16]:
# Total sales per region
region_sales = (
    sales_df.groupby('region')['final_amount_paid']
            .sum()
            .reset_index(name='total_sales')
)

# Highest‑performing region
top_region = region_sales.loc[region_sales['total_sales'].idxmax()]

# Lowest‑performing region
bottom_region = region_sales.loc[region_sales['total_sales'].idxmin()]

print("Top region:", top_region['region'], "with", top_region['total_sales'])
print("Lowest region:", bottom_region['region'], "with", bottom_region['total_sales'])


Top region: North with 9528.31
Lowest region: West with 2860.77


In [17]:
# Orders per weekday
orders_by_weekday = (
    sales_df.groupby('order_weekday')['order_id']
            .count()
            .reset_index(name='order_count')
)

# Sort by number of orders
orders_by_weekday = orders_by_weekday.sort_values('order_count', ascending=False)

# Highest weekday
top_weekday = orders_by_weekday.iloc[0]

# Lowest weekday
least_weekday = orders_by_weekday.iloc[-1]

print("Top weekday:", top_weekday['order_weekday'], "with", top_weekday['order_count'], "orders")
print("Least weekday:", least_weekday['order_weekday'], "with", least_weekday['order_count'], "orders")


Top weekday: Friday with 6 orders
Least weekday: Wednesday with 1 orders


In [18]:
# Highest shipping cost
highest_shipping = sales_df.loc[sales_df['shipping_cost'].idxmax()]

print("Highest shipping cost:", highest_shipping['shipping_cost'])
print("Order ID:", highest_shipping['order_id'])
print("Product ID:", highest_shipping['product_id'])
print("Region:", highest_shipping['region'])


Highest shipping cost: 21.57
Order ID: ORD003
Product ID: PROD001
Region: East


In [19]:
# Total shipping cost per product
shipping_by_product = (
    sales_df.groupby('product_id')['shipping_cost']
            .sum()
            .reset_index(name='total_shipping_cost')
)

# Product with highest total shipping cost
top_shipping_product = shipping_by_product.loc[
    shipping_by_product['total_shipping_cost'].idxmax()
]

print("Top product by shipping cost:", top_shipping_product['product_id'])
print("Total shipping cost:", top_shipping_product['total_shipping_cost'])


Top product by shipping cost: PROD005
Total shipping cost: 117.82
